In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from datetime import datetime, timedelta


In [2]:
# NASA API Configuration
# Replace 'YOUR_API_KEY_HERE' with your actual NASA API key
NASA_API_KEY = 'UJaP0gb7nwUokHd5AO1wgWvTTDLYGMU5oqqGnBWz'

# Base URL for NASA APIs
NASA_BASE_URL = 'https://api.nasa.gov'

print("NASA API setup complete!")
print("Don't forget to replace 'YOUR_API_KEY_HERE' with your actual API key!")


NASA API setup complete!
Don't forget to replace 'YOUR_API_KEY_HERE' with your actual API key!


# NEO (Near Earth Objects) API Functions

The NEO API provides three main endpoints:
1. **Feed**: Get asteroids based on their closest approach date to Earth
2. **Lookup**: Get details about a specific asteroid by its NASA JPL small body ID
3. **Browse**: Browse the overall asteroid dataset


In [3]:
# NEO Feed API - Get asteroids by date range
def get_neo_feed(api_key, start_date, end_date=None):
    """
    Retrieve a list of Asteroids based on their closest approach date to Earth
    
    Parameters:
    - api_key: Your NASA API key
    - start_date: Starting date in YYYY-MM-DD format
    - end_date: Ending date in YYYY-MM-DD format (optional, defaults to 7 days after start_date)
    
    Returns:
    - Dictionary containing asteroid data organized by date
    """
    url = f"{NASA_BASE_URL}/neo/rest/v1/feed"
    params = {
        'api_key': api_key,
        'start_date': start_date
    }
    
    if end_date:
        params['end_date'] = end_date
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

# Example usage:
# neo_feed = get_neo_feed(NASA_API_KEY, '2024-01-01', '2024-01-07')
# print(neo_feed)


In [4]:
# NEO Lookup API - Get specific asteroid details
def get_neo_lookup(api_key, asteroid_id):
    """
    Lookup a specific Asteroid based on its NASA JPL small body (SPK-ID) ID
    
    Parameters:
    - api_key: Your NASA API key
    - asteroid_id: Asteroid SPK-ID (NASA JPL small body ID)
    
    Returns:
    - Dictionary containing detailed asteroid information
    """
    url = f"{NASA_BASE_URL}/neo/rest/v1/neo/{asteroid_id}"
    params = {
        'api_key': api_key
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

# Example usage:
# asteroid_details = get_neo_lookup(NASA_API_KEY, 3542519)
# print(asteroid_details)


In [5]:
# Test function to verify your API key works with NEO API
def test_neo_api(api_key):
    """
    Test if your NASA API key works with the NEO API
    """
    print("Testing NEO API with your key...")
    
    # Test with a recent date range
    today = datetime.now().strftime('%Y-%m-%d')
    tomorrow = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    
    neo_data = get_neo_feed(api_key, today, tomorrow)
    
    if neo_data:
        print("✅ NEO API is working!")
        print(f"Found {neo_data.get('element_count', 0)} asteroids in the date range")
        
        # Show some basic info
        if 'near_earth_objects' in neo_data:
            dates = list(neo_data['near_earth_objects'].keys())
            print(f"Date range: {dates[0]} to {dates[-1]}")
            
            # Show first asteroid as example
            first_date = dates[0]
            if neo_data['near_earth_objects'][first_date]:
                first_asteroid = neo_data['near_earth_objects'][first_date][0]
                print(f"Example asteroid: {first_asteroid.get('name', 'Unknown')}")
    else:
        print("❌ NEO API test failed. Please check your API key.")

# Uncomment to test:
# test_neo_api(NASA_API_KEY)


# Practical Examples

Here are some practical examples of how to use the NEO API:


In [6]:
# Example 1: Get asteroids for the next week
def get_upcoming_asteroids(api_key, days_ahead=7):
    """
    Get asteroids approaching Earth in the next specified days
    """
    start_date = datetime.now().strftime('%Y-%m-%d')
    end_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')
    
    print(f"Getting asteroids from {start_date} to {end_date}...")
    
    neo_data = get_neo_feed(api_key, start_date, end_date)
    
    if neo_data:
        total_asteroids = neo_data.get('element_count', 0)
        potentially_hazardous = 0
        
        # Count potentially hazardous asteroids
        for date, asteroids in neo_data['near_earth_objects'].items():
            for asteroid in asteroids:
                if asteroid.get('is_potentially_hazardous_asteroid', False):
                    potentially_hazardous += 1
        
        print(f"\n📊 Summary:")
        print(f"Total asteroids: {total_asteroids}")
        print(f"Potentially hazardous: {potentially_hazardous}")
        
        # Show closest approach
        min_distance = float('inf')
        closest_asteroid = None
        
        for date, asteroids in neo_data['near_earth_objects'].items():
            for asteroid in asteroids:
                close_approach_data = asteroid.get('close_approach_data', [])
                for approach in close_approach_data:
                    if approach.get('close_approach_date') == date:
                        miss_distance = approach.get('miss_distance', {})
                        distance_km = float(miss_distance.get('kilometers', '0'))
                        if distance_km < min_distance:
                            min_distance = distance_km
                            closest_asteroid = {
                                'name': asteroid.get('name', 'Unknown'),
                                'distance_km': distance_km,
                                'date': date
                            }
                        break
        
        if closest_asteroid:
            print(f"\n🌍 Closest approach:")
            print(f"Asteroid: {closest_asteroid['name']}")
            print(f"Distance: {closest_asteroid['distance_km']:,.0f} km")
            print(f"Date: {closest_asteroid['date']}")
    
    return neo_data

# Uncomment to run:
# upcoming = get_upcoming_asteroids(NASA_API_KEY, 7)


In [7]:
# Example 2: Find potentially hazardous asteroids
def find_hazardous_asteroids(api_key, start_date, end_date=None):
    """
    Find all potentially hazardous asteroids in a date range
    """
    neo_data = get_neo_feed(api_key, start_date, end_date)
    
    if not neo_data:
        return None
    
    hazardous_asteroids = []
    
    for date, asteroids in neo_data['near_earth_objects'].items():
        for asteroid in asteroids:
            if asteroid.get('is_potentially_hazardous_asteroid', False):
                hazardous_asteroids.append({
                    'name': asteroid.get('name', 'Unknown'),
                    'date': date,
                    'id': asteroid.get('id', 'Unknown'),
                    'estimated_diameter': asteroid.get('estimated_diameter', {}),
                    'close_approach_data': asteroid.get('close_approach_data', [])
                })
    
    print(f"Found {len(hazardous_asteroids)} potentially hazardous asteroids:")
    for asteroid in hazardous_asteroids:
        print(f"\n⚠️  {asteroid['name']} (ID: {asteroid['id']})")
        
        # Show diameter info
        diameter = asteroid['estimated_diameter'].get('meters', {})
        if diameter:
            avg_diameter = (diameter.get('estimated_diameter_min', 0) + diameter.get('estimated_diameter_max', 0)) / 2
            print(f"   Estimated diameter: {avg_diameter:.1f} meters")
        
        # Show closest approach info
        for approach in asteroid['close_approach_data']:
            if approach.get('close_approach_date') == asteroid['date']:
                miss_distance = approach.get('miss_distance', {})
                distance_km = float(miss_distance.get('kilometers', '0'))
                print(f"   Miss distance: {distance_km:,.0f} km")
                print(f"   Relative velocity: {approach.get('relative_velocity', {}).get('kilometers_per_hour', 'Unknown')} km/h")
                break
    
    return hazardous_asteroids

# Uncomment to run:
# hazardous = find_hazardous_asteroids(NASA_API_KEY, '2024-01-01', '2024-01-07')


In [8]:
# Example 3: Get detailed information about a specific asteroid
def explore_asteroid(api_key, asteroid_id):
    """
    Get detailed information about a specific asteroid
    """
    asteroid_data = get_neo_lookup(api_key, asteroid_id)
    
    if not asteroid_data:
        print(f"Could not find asteroid with ID: {asteroid_id}")
        return None
    
    print(f"🪨 Asteroid Details: {asteroid_data.get('name', 'Unknown')}")
    print(f"NASA JPL ID: {asteroid_data.get('id', 'Unknown')}")
    print(f"Designation: {asteroid_data.get('designation', 'Unknown')}")
    
    # Physical characteristics
    diameter = asteroid_data.get('estimated_diameter', {})
    if diameter:
        meters = diameter.get('meters', {})
        if meters:
            avg_diameter = (meters.get('estimated_diameter_min', 0) + meters.get('estimated_diameter_max', 0)) / 2
            print(f"\n📏 Physical Characteristics:")
            print(f"Estimated diameter: {avg_diameter:.1f} meters")
            print(f"Diameter range: {meters.get('estimated_diameter_min', 0):.1f} - {meters.get('estimated_diameter_max', 0):.1f} meters")
    
    # Orbital data
    orbital_data = asteroid_data.get('orbital_data', {})
    if orbital_data:
        print(f"\n🛸 Orbital Data:")
        print(f"Orbital period: {orbital_data.get('orbital_period', 'Unknown')} days")
        print(f"Eccentricity: {orbital_data.get('eccentricity', 'Unknown')}")
        print(f"Inclination: {orbital_data.get('inclination', 'Unknown')} degrees")
    
    # Close approach data
    close_approaches = asteroid_data.get('close_approach_data', [])
    if close_approaches:
        print(f"\n🌍 Recent Close Approaches:")
        for approach in close_approaches[:5]:  # Show last 5 approaches
            print(f"Date: {approach.get('close_approach_date', 'Unknown')}")
            miss_distance = approach.get('miss_distance', {})
            distance_km = float(miss_distance.get('kilometers', '0'))
            print(f"  Miss distance: {distance_km:,.0f} km")
            print(f"  Relative velocity: {approach.get('relative_velocity', {}).get('kilometers_per_hour', 'Unknown')} km/h")
            print()
    
    return asteroid_data

# Uncomment to run (using the example ID from the documentation):
# asteroid_details = explore_asteroid(NASA_API_KEY, 3542519)


# How to Use This Notebook

## Step 1: Set Up Your API Key
1. Go to [https://api.nasa.gov/](https://api.nasa.gov/)
2. Fill out the form to get your free API key
3. Replace `'YOUR_API_KEY_HERE'` in the second cell with your actual API key

## Step 2: Test Your Setup
Uncomment and run the test function:
```python
test_neo_api(NASA_API_KEY)
```

## Step 3: Explore the Examples
Try out the practical examples by uncommenting the function calls:
- `get_upcoming_asteroids()` - See what's coming near Earth
- `find_hazardous_asteroids()` - Find potentially dangerous asteroids
- `explore_asteroid()` - Get detailed info about a specific asteroid

## Available NEO API Endpoints
1. **Feed**: `get_neo_feed()` - Get asteroids by date range
2. **Lookup**: `get_neo_lookup()` - Get specific asteroid details
3. **Browse**: `get_neo_browse()` - Browse all asteroids

## Rate Limits
- **Demo Key**: 30 requests per hour, 50 requests per day
- **Registered Key**: 1,000 requests per hour, unlimited per day

## Data Sources
All asteroid data comes from the NASA JPL Asteroid team at [http://neo.jpl.nasa.gov/](http://neo.jpl.nasa.gov/)

The NEO API is maintained by the SpaceRocks Team: David Greenfield, Arezu Sarvestani, Jason English and Peter Baunach.


In [9]:
def get_neo_browse(api_key, page=0, size=20):
    """
    Browse the overall Asteroid data-set
    
    Parameters:
    - api_key: Your NASA API key
    - page: Page number (default: 0)
    - size: Number of results per page (default: 20)
    
    Returns:
    - Dictionary containing paginated asteroid data
    """
    url = f"{NASA_BASE_URL}/neo/rest/v1/neo/browse"
    params = {
        'api_key': api_key,
        'page': page,
        'size': size
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

In [10]:
def get_all_asteroids(api_key, max_size_per_page=1000):
    """
    Get ALL asteroid data by automatically paginating through all available pages
    
    Parameters:
    - api_key: Your NASA API key
    - max_size_per_page: Maximum size per page (default 1000, which is the API limit)
    
    Returns:
    - List of all asteroid data from all pages
    """
    print("🚀 Fetching ALL asteroid data from NASA NEO API...")
    print("This may take a while as we're getting the complete dataset...")
    
    all_asteroids = []
    page = 0
    total_pages = None
    
    while True:
        print(f"Fetching page {page}...", end=" ")
        
        # Get data for current page
        neo_data = get_neo_browse(api_key, page=page, size=max_size_per_page)
        
        if not neo_data or 'near_earth_objects' not in neo_data:
            print("No more data available")
            break
        
        asteroids = neo_data['near_earth_objects']
        
        if not asteroids:  # Empty page means we've reached the end
            print("Reached end of data")
            break

        if len(all_asteroids) == 200:
            break
        
        # Add asteroids from this page
        all_asteroids.extend(asteroids)
        print(f"Got {len(asteroids)} asteroids (Total so far: {len(all_asteroids)})")
        
        # Check if we have page information
        if 'page' in neo_data:
            current_page_info = neo_data['page']
            if 'total_pages' in current_page_info:
                total_pages = current_page_info['total_pages']
                print(f"Total pages available: {total_pages}")
        
        # Move to next page
        page += 1
        
        # Safety check to prevent infinite loops
        if page > 1000:  # Reasonable upper limit
            print("Reached safety limit of 1000 pages")
            break
    
    print(f"\n✅ Complete! Retrieved {len(all_asteroids)} asteroids from {page} pages")
    return all_asteroids


In [11]:
# Convert ALL asteroid data to DataFrame
def neo_browse_to_dataframe(api_key, max_size_per_page=1000):
    """
    Get ALL asteroid data and convert it to a pandas DataFrame
    
    Parameters:
    - api_key: Your NASA API key
    - max_size_per_page: Maximum size per page (default 1000)
    
    Returns:
    - pandas DataFrame with ALL asteroid data
    """
    print("🔄 Getting ALL asteroid data and converting to DataFrame...")
    
    # Get all asteroid data
    all_asteroids =get_all_asteroids(api_key, max_size_per_page)
    
    if not all_asteroids:
        print("No asteroid data available")
        return None
    
    print(f"\n📊 Converting {len(all_asteroids)} asteroids to DataFrame...")
    
    # Prepare data for DataFrame (same logic as before but for all data)
    df_data = []
    
    for i, asteroid in enumerate(all_asteroids):
        if i % 1000 == 0:  # Progress indicator
            print(f"Processing asteroid {i+1}/{len(all_asteroids)}...")
        
        # Extract basic information
        row = {
            'id': asteroid.get('id', ''),
            'name': asteroid.get('name', ''),
            'designation': asteroid.get('designation', ''),
            'absolute_magnitude_h': asteroid.get('absolute_magnitude_h', ''),
            'is_potentially_hazardous': asteroid.get('is_potentially_hazardous_asteroid', False),
            'is_sentry_object': asteroid.get('is_sentry_object', False)
        }
        
        # Extract diameter information
        estimated_diameter = asteroid.get('estimated_diameter', {})
        if estimated_diameter:
            meters = estimated_diameter.get('meters', {})
            if meters:
                row['diameter_min_m'] = meters.get('estimated_diameter_min', '')
                row['diameter_max_m'] = meters.get('estimated_diameter_max', '')
                row['diameter_avg_m'] = (meters.get('estimated_diameter_min', 0) + meters.get('estimated_diameter_max', 0)) / 2
            else:
                row['diameter_min_m'] = ''
                row['diameter_max_m'] = ''
                row['diameter_avg_m'] = ''
        else:
            row['diameter_min_m'] = ''
            row['diameter_max_m'] = ''
            row['diameter_avg_m'] = ''
        
        # Extract orbital data
        orbital_data = asteroid.get('orbital_data', {})
        if orbital_data:
            row['orbital_period_days'] = orbital_data.get('orbital_period', '')
            row['eccentricity'] = orbital_data.get('eccentricity', '')
            row['inclination_deg'] = orbital_data.get('inclination', '')
            row['perihelion_distance_au'] = orbital_data.get('perihelion_distance', '')
            row['aphelion_distance_au'] = orbital_data.get('aphelion_distance', '')
        else:
            row['orbital_period_days'] = ''
            row['eccentricity'] = ''
            row['inclination_deg'] = ''
            row['perihelion_distance_au'] = ''
            row['aphelion_distance_au'] = ''
        
        # Extract close approach data (most recent)
        close_approach_data = asteroid.get('close_approach_data', [])
        if close_approach_data:
            # Get the most recent close approach
            latest_approach = close_approach_data[0]
            row['last_close_approach_date'] = latest_approach.get('close_approach_date', '')
            row['last_miss_distance_km'] = float(latest_approach.get('miss_distance', {}).get('kilometers', '0'))
            row['last_relative_velocity_kmh'] = float(latest_approach.get('relative_velocity', {}).get('kilometers_per_hour', '0'))
            row['orbiting_body'] = latest_approach.get('orbiting_body', '')
        else:
            row['last_close_approach_date'] = ''
            row['last_miss_distance_km'] = ''
            row['last_relative_velocity_kmh'] = ''
            row['orbiting_body'] = ''
        
        df_data.append(row)
    
    print("Creating DataFrame...")
    
    # Create DataFrame
    df = pd.DataFrame(df_data)
    
    # Convert numeric columns
    numeric_columns = ['absolute_magnitude_h', 'diameter_min_m', 'diameter_max_m', 'diameter_avg_m', 
                      'orbital_period_days', 'eccentricity', 'inclination_deg', 
                      'perihelion_distance_au', 'aphelion_distance_au', 'last_miss_distance_km', 
                      'last_relative_velocity_kmh']
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert boolean columns
    boolean_columns = ['is_potentially_hazardous', 'is_sentry_object']
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].astype(bool)
    
    print(f"✅ Complete! Created DataFrame with {len(df)} asteroids")
    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    return df

# Example usage:
# df_all = all_asteroids_to_dataframe(NASA_API_KEY)


In [12]:
df = neo_browse_to_dataframe(NASA_API_KEY)

🔄 Getting ALL asteroid data and converting to DataFrame...
🚀 Fetching ALL asteroid data from NASA NEO API...
This may take a while as we're getting the complete dataset...
Fetching page 0... Got 20 asteroids (Total so far: 20)
Total pages available: 2034
Fetching page 1... Got 20 asteroids (Total so far: 40)
Total pages available: 2034
Fetching page 2... Got 20 asteroids (Total so far: 60)
Total pages available: 2034
Fetching page 3... Got 20 asteroids (Total so far: 80)
Total pages available: 2034
Fetching page 4... Got 20 asteroids (Total so far: 100)
Total pages available: 2034
Fetching page 5... Got 20 asteroids (Total so far: 120)
Total pages available: 2034
Fetching page 6... Got 20 asteroids (Total so far: 140)
Total pages available: 2034
Fetching page 7... Got 20 asteroids (Total so far: 160)
Total pages available: 2034
Fetching page 8... Got 20 asteroids (Total so far: 180)
Total pages available: 2034
Fetching page 9... Got 20 asteroids (Total so far: 200)
Total pages availabl

In [20]:
df = df[df['is_potentially_hazardous'] == True]
df.to_json('asteroid_data.json')
df.to_csv('asteroid_data.csv')